In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import datasets, layers, models
from keras.preprocessing.text import Tokenizer


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from keras.models import Model
from keras.layers import *
from keras.utils.np_utils import to_categorical
import re



# 데이터 불러오기

In [ ]:
from gensim import models

ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin.gz')

df = pd.read_excel('ratings_train.xlsx')
doc = list(df['document'])
labels = list(df['label'])

# 한글만 추출

In [ ]:
only_korean= []
error_list = []
for idx, sentence in enumerate(doc):
    try:
        only_korean.append(re.findall('[가-힣]+', sentence))
    except:
        only_korean.append('0')
        error_list.append(idx)

# 토크나이즈

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(only_korean)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
X_encoded = tokenizer.texts_to_sequences(only_korean)
max_len = max(len(I) for I in X_encoded)

In [ ]:
X_train = pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train = np.array(labels)

In [ ]:
import numpy as np

VOCAB_SIZE = len(tokenizer.index_word) + 1
EMBEDDING_DIM = 300

embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

for word, idx in tokenizer.word_index.items():
    embedding_vector = ko_model.wv.word_vec(word)
    if embedding_vector is not None:
        embedding_matrix[idx] = embedding_vector
        
embedding_matrix.shape

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=max_len, trainable=False),
  tf.keras.layers.Conv1D(filters = 512, kernel_size=4, activation='relu'),  
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(6, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train, y_train, batch_size = 100, epochs=5, validation_split=0.2)